In [1]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
#os.environ.pop('CUDA_VISIBLE_DEVICES')
#os.unsetenv('CUDA_VISIBLE_DEVICES')

In [17]:
# Standard imports
import sys
import itertools
import glob

# Package imports
import numpy as np
import pandas as pd
from keras import models, layers
import matplotlib.pyplot as plt

# Local imports
#import dutils
#import drawing

%matplotlib inline
np.random.seed(2341)

In [18]:
hits_row_names = ["event_id", "track_id", "i_r", "i_phi", "x", "y"]
particles_row_names = ["event_id", "p_id", "pt", "phi", "vx", "vy"]
n_r_bins = 9
n_phi_bins = 1000
max_tracks = 25
pt_scale = 2*np.pi/20000 # for loss function

def get_phi(x, y):
    return np.arctan2(y, x)

def discretize(evt, phiwidth):
    """
    evt: pandas dataframe holding hit information for one event
    phiwidth: size of pixel in phi direction
    
    Converts the hit information in the event into a numpy array with
    hits represented by 1's.
    """
    
    r_grid = np.arange(n_r_bins)
    phi_grid = np.arange(0., 2*np.pi, phiwidth)
    image = np.zeros((len(r_grid),len(phi_grid), 1))
    try:
        for hit in evt.itertuples():
            ir = hit[hits_row_names.index('i_r')]
            x = hit[hits_row_names.index('x')]
            y = hit[hits_row_names.index('y')]
            phi = get_phi(x, y)
            iphi = int((phi/phiwidth))
            image[ir,iphi,0] = 1
    except AttributeError:
        # This occurs if the event has only one hit (rare), in which case evt
        # is a Series, not a DataFrame.  Deal with this separately.
        print "Encountered event with only one hit:",evt
        
    return image

def get_targets(evt, max_tracks):
    """
    evt: pandas dataframe holding particle info for one event
    
    Gets the pt and phi of each particle in the event and returns them in a numpy array,
    scaled to an appropriate scale.
    """
    particles = np.zeros((max_tracks, 2))
    weights = np.zeros(max_tracks)
    try:
        for i,particle in enumerate(evt.itertuples()):
            if i >= max_tracks:
                break
            particles[i, 0] = particle[particles_row_names.index('pt')]*pt_scale
            particles[i, 1] = particle[particles_row_names.index('phi')]
            weights[i] = 1
        particles[:i+1] = particles[particles[:i+1,1].argsort()] # sort by second column (phi)
    except AttributeError:
        # This occurs if the event has only one particle, in which case evt is
        # a Series, not a DataFrame.  Deal with this separately.
        particles[0, 0] = evt['pt']*pt_scale
        particles[0, 1] = evt['phi']
        weights[0] = 1
    
    return particles, weights

In [19]:
def gen_single_hits(hit_files):
    cur_file = 0
    num_files = len(hit_files)
    while True:
        df = pd.read_csv(hit_files[cur_file], header=None, names=hits_row_names, index_col=hits_row_names[0])
        event_nums = sorted(df.index.unique())
        for event_num in event_nums:
            yield discretize(df.loc[event_num], phiwidth=2*np.pi/n_phi_bins)
        cur_file += 1
        if cur_file >= num_files:
            cur_file = 0

def gen_single_particles(particle_files, max_tracks):
    cur_file = 0
    num_files = len(particle_files)
    while True:
        df = pd.read_csv(particle_files[cur_file], header=None, names=particles_row_names, index_col=particles_row_names[0])
        event_nums = sorted(df.index.unique())
        for event_num in event_nums:
            yield get_targets(df.loc[event_num], max_tracks)
        cur_file += 1
        if cur_file >= num_files:
            cur_file = 0
    
def generate_data(batch_size, hit_files, particle_files, max_tracks=max_tracks):
    gen_hits = gen_single_hits(hit_files)
    gen_particles = gen_single_particles(particle_files, max_tracks)
    while True:
        batch_events = np.array([evt for evt in itertools.islice(gen_hits, batch_size)])
        batch_targets = np.zeros((batch_size, max_tracks, 2))
        batch_weights = np.zeros((batch_size, max_tracks))
        for i,(particles, weights) in enumerate(itertools.islice(gen_particles, batch_size)):
            batch_targets[i] = particles
            batch_weights[i] = weights
        yield batch_events, batch_targets, batch_weights

In [20]:
hit_files = sorted(glob.glob("/bigdata/share/TkRamp/hits_*.csv"))
particle_files = sorted(glob.glob("/bigdata/share/TkRamp/particles_*.csv"))
generator = generate_data(256, hit_files, particle_files)

In [21]:
hit_files

[]

In [11]:
# Model
def build_model_1(max_tracks=max_tracks):
    internal = 150
    input_layer = layers.Input(shape=(n_r_bins, n_phi_bins, 1))
    
    layer1 = layers.Convolution2D(8, 2, 6, border_mode='same')(input_layer)
    layer1 = layers.Activation('relu')(layer1)
    layer1 = layers.Convolution2D(8, 2, 6, border_mode='same')(layer1)
    layer1 = layers.Activation('relu')(layer1)
    layer1 = layers.MaxPooling2D(pool_size=(1,4))(layer1)
    layer1 = layers.Convolution2D(16, 2, 6, border_mode='same')(layer1)
    layer1 = layers.Activation('relu')(layer1)
    layer1 = layers.Convolution2D(16, 2, 6, border_mode='same')(layer1)
    layer1 = layers.Activation('relu')(layer1)
    layer1 = layers.Flatten()(layer1)
    
    layer2 = layers.Convolution2D(8, 2, 12, border_mode='same')(input_layer)
    layer2 = layers.Activation('relu')(layer2)
    layer2 = layers.Convolution2D(8, 2, 12, border_mode='same')(layer2)
    layer2 = layers.Activation('relu')(layer2)
    layer2 = layers.MaxPooling2D(pool_size=(2,4))(layer2)
    layer2 = layers.Convolution2D(16, 2, 6, border_mode='same')(layer2)
    layer2 = layers.Activation('relu')(layer2)
    layer2 = layers.Convolution2D(16, 2, 6, border_mode='same')(layer2)
    layer2 = layers.Activation('relu')(layer2)
    layer2 = layers.Flatten()(layer2)
    
    layer3 = layers.Convolution2D(8, 4, 12, border_mode='same')(input_layer)
    layer3 = layers.Activation('relu')(layer3)
    layer3 = layers.Convolution2D(8, 4, 12, border_mode='same')(layer3)
    layer3 = layers.Activation('relu')(layer3)
    layer3 = layers.Flatten()(layer3)
    
    layer = layers.merge([layer1, layer2, layer3], mode='concat', concat_axis=1)
    layer = layers.Dense(internal)(layer)
    layer = layers.RepeatVector(max_tracks)(layer)
    layer = layers.LSTM(internal, return_sequences=True)(layer)
    output_layer = layers.TimeDistributed(layers.Dense(2))(layer)
    model = models.Model(input=input_layer, output=output_layer)
    model.compile(loss='mean_squared_error', optimizer='Adam', sample_weight_mode="temporal")
    return model

In [12]:
model_1 = build_model_1()

ValueError: Negative dimension size caused by subtracting 4 from 1 for 'MaxPool_1' (op: 'MaxPool') with input shapes: [?,1000,1,8].

In [19]:
model_1.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 1, 9, 1000)    0                                            
____________________________________________________________________________________________________
convolution2d_11 (Convolution2D) (None, 8, 9, 1000)    104         input_2[0][0]                    
____________________________________________________________________________________________________
convolution2d_15 (Convolution2D) (None, 8, 9, 1000)    200         input_2[0][0]                    
____________________________________________________________________________________________________
activation_11 (Activation)       (None, 8, 9, 1000)    0           convolution2d_11[0][0]           
___________________________________________________________________________________________

In [ ]:
%time model_1.fit_generator(generator, samples_per_epoch=96000, nb_epoch=3)

Epoch 1/3
96000/96000 [==============================] - 394s - loss: 1.6117   
Epoch 2/3
43008/96000 [============>.................] - ETA: 216s - loss: 1.2180

KeyboardInterrupt: 

In [ ]:
%time model_1.fit_generator(generator, samples_per_epoch=96000, nb_epoch=3)

Epoch 1/3
92672/96000 [===========================>..] - ETA: 13s - loss: 1.0390

In [ ]:
evt, truth, weight = generator.next()
#pred = model_1.predict(np.array([evt[0]]))
pred = model_1.predict(evt)
print "Model prediction"
print pred[0]
print "Target"
print truth[0]

In [ ]:
diff = []
relative_diff = []
for i in range( truth.shape[0]):
    t = truth[i]
    p = pred[i]
    one = np.where(t[:,0] !=0 )
    #print t[one,0]
    #print p[one,0]
    d  = (t[one,0] - p[one,0])
    rd = d /t[one,0] 
    diff.extend(d.ravel())
    relative_diff.extend(rd.ravel())


plt.hist(diff )
plt.xlabel('truth-prediction')
plt.show()

plt.hist(relative_diff )
plt.xlabel('truth-prediction/prediction')
plt.show()